In [1]:
# we want one table with all the steering results
# 1. explicit, implicit
# different vector explicit 
# universal vector  

import pandas as pd

In [22]:
all_vecs = pd.read_csv('/scratch/gpfs/vv7118/projects/localization-gap/analysis/all_steering_results_per_task_culture.csv')

#### Get the best steering params for each vector 

In [23]:
steering_params = pd.DataFrame()

# per-culture (en) - select the best perform steering params for the entire culture. 
per_culture_en = all_vecs[all_vecs['vector'] == 'per-culture (en)']
per_culture_en = per_culture_en[(per_culture_en['translated'] == False) | (per_culture_en['country'] == 'United States')]
per_culture_en = per_culture_en.groupby(['layer', 'alpha', 'country'])['local_ans_steer'].mean().reset_index()
best_params = per_culture_en.loc[per_culture_en.groupby('country')['local_ans_steer'].idxmax()]
best_params['vector'] = 'per-culture (en)'
best_params.drop(columns=['local_ans_steer'], inplace=True)
steering_params = pd.concat([steering_params, best_params])


# per-culture (translated) - select the best perform steering params for the entire culture. 
per_culture_en = all_vecs[all_vecs['vector'] == 'per-culture (translated)']
per_culture_en = per_culture_en[(per_culture_en['translated'] == True) | (per_culture_en['country'] == 'United States')]
per_culture_en = per_culture_en.groupby(['layer', 'alpha', 'country'])['local_ans_steer'].mean().reset_index()
best_params = per_culture_en.loc[per_culture_en.groupby('country')['local_ans_steer'].idxmax()]
best_params['vector'] = 'per-culture (translated)'
best_params.drop(columns=['local_ans_steer'], inplace=True)
steering_params = pd.concat([steering_params, best_params])


# names (en) - select the best perfomring steering vector for each culture only on that task 
names_en = all_vecs[all_vecs['vector'] == 'names (en)']
names_en = names_en[(names_en['translated'] == False) | (names_en['country'] == 'United States')]
names_en = names_en.groupby(['layer', 'alpha', 'country'])['local_ans_steer'].mean().reset_index()
best_params = names_en.loc[names_en.groupby('country')['local_ans_steer'].idxmax()]
best_params['vector'] = 'names (en)'
best_params.drop(columns=['local_ans_steer'], inplace=True)
steering_params = pd.concat([steering_params, best_params])

# universal vector (en) - select the best performing steering vector for each culture only on that task 
universal_en = all_vecs[all_vecs['vector'] == 'held-out universal (en)']
universal_en = universal_en[(universal_en['translated'] == False) | (universal_en['country'] == 'United States')]
universal_en = universal_en.groupby(['layer', 'alpha', 'country'])['local_ans_steer'].mean().reset_index()
best_params = universal_en.loc[universal_en.groupby('country')['local_ans_steer'].idxmax()]
best_params['vector'] = 'held-out universal (en)'
best_params.drop(columns=['local_ans_steer'], inplace=True)
steering_params = pd.concat([steering_params, best_params])

# universal vector (translated) - select the best performing steering vector for each culture only on that task 
universal_translated = all_vecs[all_vecs['vector'] == 'held-out universal (translated)']
universal_translated = universal_translated[(universal_translated['translated'] == True) | (universal_translated['country'] == 'United States')]
universal_translated = universal_translated.groupby(['layer', 'alpha', 'country'])['local_ans_steer'].mean().reset_index()
best_params = universal_translated.loc[universal_translated.groupby('country')['local_ans_steer'].idxmax()]
best_params['vector'] = 'held-out universal (translated)'
best_params.drop(columns=['local_ans_steer'], inplace=True)
steering_params = pd.concat([steering_params, best_params])


In [29]:
steering_params

,layer,alpha,country,vector
255,27,2,Bangladesh,per-culture (en)
211,25,1,France,per-culture (en)
217,25,2,Russia,per-culture (en)
198,24,2,Turkey,per-culture (en)
159,22,2,United States,per-culture (en)
215,25,2,Bangladesh,per-culture (translated)
216,25,2,France,per-culture (translated)
257,27,2,Russia,per-culture (translated)
138,21,2,Turkey,per-culture (translated)
159,22,2,United States,per-culture (translated)


In [30]:
all_vecs_table1 = pd.merge(all_vecs, steering_params, on=['layer', 'alpha', 'country', 'vector'], how='inner')

In [31]:
all_vecs_table1

,vector,alpha,layer,country,translated,subtask,local_ans_steer,local_ans_no_steer,local_ans_hinted
0,per-culture (en),1,25,France,False,cities,0.782857,0.491429,1.000000
1,per-culture (en),1,25,France,False,culturebench,0.794872,0.717949,0.948718
2,per-culture (en),1,25,France,False,culturedistil,0.603896,0.428571,0.863636
3,per-culture (en),1,25,France,False,names,0.665094,0.584906,0.830189
4,per-culture (en),1,25,France,True,cities,0.508571,0.451429,1.000000
...,...,...,...,...,...,...,...,...,...
103,per-culture (translated),2,27,Russia,False,names,0.818841,0.579710,0.934783
104,per-culture (translated),2,27,Russia,True,cities,0.822857,0.382857,1.000000
105,per-culture (translated),2,27,Russia,True,culturebench,0.678571,0.666667,0.738095
106,per-culture (translated),2,27,Russia,True,culturedistil,0.721519,0.645570,0.816456


In [33]:
table1_vecs = ['per-culture (en)', 'per-culture (translated)', 'names (en)']

table1_df = all_vecs_table1[all_vecs_table1['vector'].isin(table1_vecs)]
table1_df = table1_df[table1_df['translated'] == True]

table1_gp = table1_df.groupby(['vector', 'subtask'])[['local_ans_steer',]].mean().reset_index()
table1_gp = table1_gp.pivot(index='subtask', columns='vector', values='local_ans_steer')

In [34]:
table1_context = table1_df.groupby(['subtask', 'vector'])[['local_ans_no_steer', 'local_ans_hinted']].mean().reset_index()

# Merge the no_steer and hinted columns from table1_context into table1_gp
for col in ['local_ans_no_steer', 'local_ans_hinted']:
    table1_gp[col] = table1_context.groupby('subtask')[col].mean()
table1_gp = table1_gp.sort_values(by='subtask')

for col in ['names (en)', 'per-culture (en)', 'per-culture (translated)', 'local_ans_no_steer', 'local_ans_hinted']:
    table1_gp[col] = table1_gp[col].apply(lambda x: f"{x:.3f}")

table1_gp = table1_gp[[ 'per-culture (en)', 'per-culture (translated)', 'names (en)', 'local_ans_no_steer', 'local_ans_hinted']]
table1_gp.columns = ['English', 'Translated', 'Names', 'Implicit', 'Explicit']
print(table1_gp.to_latex(index=True))


\begin{tabular}{llllll}
\toprule
 & English & Translated & Names & Implicit & Explicit \\
subtask &  &  &  &  &  \\
\midrule
cities & 0.685 & 0.797 & 0.667 & 0.426 & 1.000 \\
culturebench & 0.784 & 0.769 & 0.769 & 0.752 & 0.860 \\
culturedistil & 0.624 & 0.606 & 0.635 & 0.557 & 0.786 \\
names & 0.721 & 0.737 & 0.748 & 0.607 & 0.909 \\
\bottomrule
\end{tabular}



In [4]:
# table 1: per-culture vector (en), per-culture vector (tr), names vector, explicit, implicit
# - rows are tasks 

# universal vector: held-out (en), held-out (tr), explicit, implicit
# - rows are languages 

,vector,alpha,layer,country,translated,local_ans_steer,local_ans_no_steer,local_ans_hinted
0,per-culture (en),2,21,Bangladesh,False,0.769896,0.543253,0.955017
1,per-culture (en),2,21,Bangladesh,True,0.738754,0.628028,0.894464
2,per-culture (en),1,25,France,False,0.693103,0.524138,0.898276
3,per-culture (en),2,21,France,True,0.612069,0.546552,0.884483
4,per-culture (en),2,25,Russia,False,0.872072,0.497297,0.926126
...,...,...,...,...,...,...,...,...
58,held-out universal (translated),2,27,Russia,False,0.556757,0.497297,0.926126
59,held-out universal (translated),2,25,Russia,True,0.690090,0.531532,0.877477
60,held-out universal (translated),2,27,Turkey,False,0.634286,0.548571,0.944286
61,held-out universal (translated),2,27,Turkey,True,0.680000,0.558571,0.918571


### Implicit vector table

In [ ]:
df = pd.read_csv('/scratch/gpfs/vv7118/projects/localization-gap/analysis/best_steering_per_task_culture.csv')

In [13]:
implicit_df = pd.read_csv('best_steering_res_implicit_per_culture.csv')



In [15]:
explicit_steer = df[(df['vector'] == 'per-culture (en)') & (df['translated'] == True)]

In [17]:
explicit_steer = explicit_steer.sort_values(by='country')

,vector,alpha,layer,country,translated,local_ans_steer,local_ans_no_steer,local_ans_hinted
1,per-culture (en),2,21,Bangladesh,True,0.738754,0.628028,0.894464
3,per-culture (en),2,21,France,True,0.612069,0.546552,0.884483
5,per-culture (en),2,25,Russia,True,0.792793,0.531532,0.877477
7,per-culture (en),2,21,Turkey,True,0.762857,0.558571,0.918571
8,per-culture (en),2,22,United States,True,0.695090,0.523686,0.882860


In [20]:
implicit_df = implicit_df[['alpha','layer','country','local_ans_steer','local_ans_no_steer']]
implicit_df = implicit_df.sort_values(by='country')
implicit_df['explicit_max'] = explicit_steer.reset_index()['local_ans_steer']




In [23]:
implicit_df.columns = ['alpha', 'layer', 'country', 'implicit steering', 'no steering', 'explicit steering']
for col in ['implicit steering', 'no steering', 'explicit steering']:
    implicit_df[col] = implicit_df[col].apply(lambda x: f"{x:.3f}")



In [26]:
implicit_df = implicit_df[['alpha', 'layer', 'country', 'implicit steering', 'explicit steering', 'no steering']]
print(implicit_df.to_latex(index=False))

\begin{tabular}{rrllll}
\toprule
alpha & layer & country & implicit steering & explicit steering & no steering \\
\midrule
1 & 25 & Bangladesh & 0.696 & 0.739 & 0.628 \\
2 & 26 & France & 0.595 & 0.612 & 0.547 \\
1 & 25 & Russia & 0.614 & 0.793 & 0.532 \\
2 & 23 & Turkey & 0.654 & 0.763 & 0.559 \\
1 & 25 & United States & 0.621 & 0.695 & 0.518 \\
\bottomrule
\end{tabular}



### Universal vector table
- rows are languages
- columns are held-out universal (en), held-out universal (translated), explicit, implicit 


In [25]:
univ_vecs = ['per-culture (en)', 'per-culture (translated)', 'names (en)','held-out universal (en)', 'held-out universal (translated)']

In [30]:
uni_df = df[df['vector'].isin(univ_vecs)]
uni_df = uni_df[uni_df['translated'] == True]
uni_df_gp= uni_df.groupby(['vector', 'country'])['local_ans_steer'].mean().reset_index()
uni_df_gp = uni_df_gp.pivot(index='country', columns='vector', values='local_ans_steer').sort_values(by='country')

# other cols 
other_cols = ['local_ans_no_steer', 'local_ans_hinted']
other_cols_df = uni_df.groupby('country')[other_cols].mean().reset_index().sort_values(by='country')

for col in other_cols:
    uni_df_gp[col] = other_cols_df[col].values





In [31]:
uni_df_gp = uni_df_gp[univ_vecs+other_cols]



In [34]:
for col in univ_vecs+other_cols:
    uni_df_gp[col] = uni_df_gp[col].apply(lambda x: f"{x:.3f}")

print(uni_df_gp.to_latex())

\begin{tabular}{llllllll}
\toprule
vector & per-culture (en) & per-culture (translated) & names (en) & held-out universal (en) & held-out universal (translated) & local_ans_no_steer & local_ans_hinted \\
country &  &  &  &  &  &  &  \\
\midrule
Bangladesh & 0.726 & 0.808 & 0.755 & 0.690 & 0.695 & 0.621 & 0.883 \\
France & 0.676 & 0.693 & 0.717 & 0.671 & 0.656 & 0.597 & 0.895 \\
Russia & 0.787 & 0.757 & 0.802 & 0.729 & 0.706 & 0.551 & 0.858 \\
Turkey & 0.769 & 0.727 & 0.781 & 0.700 & 0.678 & 0.560 & 0.913 \\
United States & 0.768 & 0.768 & 0.775 & 0.742 & 0.722 & 0.599 & 0.895 \\
\bottomrule
\end{tabular}

